In [51]:
import os, yaml, re

def change_root(root: str, to_change: str) -> str:
    root_direc_split, to_change_split = root.split(os.path.sep), re.split("/|\\\\", to_change)
    root_direc_last = root_direc_split[-1]
    to_change_last_index = to_change_split.index(root_direc_last)
    new = os.path.sep.join(root_direc_split + to_change_split[to_change_last_index+1:])
    return new

def update_yaml_artifact_path(mlrun_direc: str) -> None:
    """
    Will search through the provided directory and update the 'artifact_uri' within the yaml files it contains.

    This is to fix issues with MLFLOW where absolute paths are used for all artifact URIs.
    """
    assert os.path.isdir(mlrun_direc), f"Input path {mlrun_direc} must be a directory."
    for root, dirs, files in os.walk(mlrun_direc):
        for f in files:
            if os.path.splitext(f)[1] in [".yaml", ".yml"]:
                this_path = os.path.join(root, f)
                yml = yaml.safe_load(open(this_path, 'r'))
                if 'artifact_uri' in yml.keys(): 
                    artifact_path = yml['artifact_uri']
                    yml['artifact_uri'] = change_root(root, artifact_path)
                elif 'artifact_location' in yml.keys():
                    artifact_path = yml['artifact_location']
                    yml['artifact_location'] = change_root(root, artifact_path)
                else: continue
                yaml.dump(yml, open(this_path, "w"))

In [52]:
update_yaml_artifact_path("/mnt/d/mlruns")